In [1]:
import pandas as pd
import torch

class RE_Dataset(torch.utils.data.Dataset):
  """ Dataset 구성을 위한 class."""
  def __init__(self, pair_dataset, labels):
    self.pair_dataset = pair_dataset
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

def preprocessing_dataset(dataset):
    """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경"""
    labels = []
    sentences = []
    for i,j in zip(dataset['label'], dataset['sentence']):
        labels.append(i)
        sentences.append(j)
    out_dataset = pd.DataFrame({'label':dataset['label'], 'sentence':dataset['sentence']})
    return out_dataset

def load_data(dataset_dir):
    """csv 파일을 경로에 맡게 불러 옵니다."""
    pd_dataset = pd.read_csv(dataset_dir, encoding='cp949')
    dataset = preprocessing_dataset(pd_dataset)
    return dataset

def tokenized_dataset(dataset, tokenizer):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for s in dataset['sentence']: # 데이터셋의 엔티티 두개를 추출
    concat_entity.append(s)
  tokenized_sentences = tokenizer(
      concat_entity,
      #list(dataset['sentence']),
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=256,
      add_special_tokens=True,
      )
  return tokenized_sentences

In [2]:
from sklearn.preprocessing import LabelEncoder

label_list = ['분노', '슬픔','불안','상처','당황','기쁨']

encoder = LabelEncoder()
encoder.fit(label_list)

print(encoder.transform(label_list))


[2 5 3 4 1 0]


In [49]:
def label_to_num(label):
    num_label = encoder.transform(label)
    return num_label

def num_to_label(label):
    origin_label = encoder.inverse_transform(label)
    return origin_label

In [4]:
import sklearn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['분노', '슬픔','불안','상처','당황','기쁨']
    #no_relation_label_idx = label_list.index("no_relation")
    label_indices = list(range(len(label_list)))
    #label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0

def klue_re_auprc(probs, labels):
    """KLUE-RE AUPRC (with no_relation)"""
    labels = np.eye(6)[labels]

    score = np.zeros((6,))
    for c in range(6):
        targets_c = labels.take([c], axis=1).ravel()
        preds_c = probs.take([c], axis=1).ravel()
        precision, recall, _ = sklearn.metrics.precision_recall_curve(
            targets_c, preds_c)
        score[c] = sklearn.metrics.auc(recall, precision)
    return np.average(score) * 100.0


def compute_metrics(pred):
    """ validation을 위한 metrics function """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = pred.predictions

    # calculate accuracy using sklearn's function
    f1 = klue_re_micro_f1(preds, labels)
    auprc = klue_re_auprc(probs, labels)
    acc = accuracy_score(labels, preds)


    return {
        'micro f1 score': f1,
        'auprc': auprc,
        'accuracy': acc,
    }


In [5]:
import torch
import random
import os
import numpy as np

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


In [16]:
dataset = load_data('./dataset.csv')
dataset["label"]
print(dataset)

      label                                           sentence
0        기쁨                          아내가 드디어 출산하게 되어서 정말 신이 나.
1        불안            당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2        당황            고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3        기쁨      재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4        기쁨                       빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.
...     ...                                                ...
40874    불안  같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...
40875    기쁨                  지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어.
40876    슬픔          엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.
40877    기쁨            이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야.
40878    불안                친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.

[40879 rows x 2 columns]


In [23]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split


def train():
    seed_everything(42)
    # load model and tokenizer
    MODEL_NAME = 'klue/bert-base'
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # load dataset
    dataset = load_data('./dataset.csv')
    target = label_to_num(dataset["label"].values)
    train_dataset, dev_dataset = train_test_split(
        dataset, test_size=0.15, shuffle=True, stratify=target,
    )

    train_label = label_to_num(train_dataset['label'].values)
    dev_label = label_to_num(dev_dataset['label'].values)

    # tokenizing dataset
    tokenized_train = tokenized_dataset(train_dataset, tokenizer)
    tokenized_dev = tokenized_dataset(dev_dataset, tokenizer)

    # make dataset for pytorch.
    RE_train_dataset = RE_Dataset(tokenized_train, train_label)
    RE_dev_dataset = RE_Dataset(tokenized_dev, dev_label)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    print(device)

    # setting model hyperparameter
    model_config = AutoConfig.from_pretrained(MODEL_NAME)
    model_config.num_labels = 6

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, config=model_config)
    print(model.config)
    model.parameters
    model.to(device)

    # 사용한 option 외에도 다양한 option들이 있습니다.
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        save_total_limit=5,              # number of total save model.
        save_steps=500,                 # model saving step.
        num_train_epochs=10,              # total number of training epochs
        learning_rate=1e-4,               # learning_rate
        # batch size per device during training
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,   # batch size for evaluation
        warmup_ratio=0.1,
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=100,              # log saving step.
        evaluation_strategy='steps',  # evaluation strategy to adopt during training
        # `no`: No evaluation during training.
        # `steps`: Evaluate every `eval_steps`.
        # `epoch`: Evaluate every end of epoch.
        eval_steps=500,            # evaluation step.
        metric_for_best_model='eval_micro f1 score',  # eval_micro f1 score
        load_best_model_at_end=True,
        lr_scheduler_type='cosine',  # default: linear
    )

    trainer = Trainer(
        # the instantiated 🤗 Transformers model to be trained
        model=model,
        args=training_args,                  # training arguments, defined above
        train_dataset=RE_train_dataset,         # training dataset
        eval_dataset=RE_dev_dataset,             # evaluation dataset
        compute_metrics=compute_metrics         # define metrics function
    )

    # trainer.hyperparameter_search(direction="maximize", hp_space=my_hp_space_ray)
    # train model
    trainer.train()

    model.save_pretrained(f'./best_model/')

In [24]:
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("cuda empty cache!!")

cuda empty cache!!


In [25]:
train()

cuda:0


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



Step,Training Loss,Validation Loss,Micro f1 score,Auprc,Accuracy,Runtime,Samples Per Second
500,1.032300,1.075343,61.692759,68.550034,0.616928,10.289300,595.956000
1000,0.937700,1.003533,64.155251,71.447631,0.641553,10.454600,586.536000
1500,0.761100,1.050495,64.171559,70.751000,0.641716,10.288800,595.986000
2000,0.547900,1.149174,65.410959,69.971622,0.654110,10.279100,596.549000
2500,0.344000,1.339058,64.856491,69.484260,0.648565,10.282400,596.359000
3000,0.212800,1.508211,65.117417,68.712111,0.651174,10.275000,596.789000
3500,0.124900,1.749596,65.231572,68.146820,0.652316,10.421700,588.390000
4000,0.070600,1.938252,64.774951,67.704602,0.647750,11.133000,550.793000
4500,0.045200,2.046523,65.362035,67.753900,0.653620,10.334100,593.375000
5000,0.029400,2.097542,65.215264,67.841107,0.652153,11.554000,530.727000


In [29]:
# top-k개 추론 결과를 보여주자

def tokenizing(sentence, tokenizer):
    """ tokenizer에 따라 sentence를 tokenizing 합니다."""
    tokenized_sentence = tokenizer(
        sentence,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256,
        add_special_tokens=True,
      )
        
    return tokenized_sentence

In [33]:
MODEL_NAME = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
a = tokenizing("안녕하세요.", tokenizer)

In [35]:
print(a)
print(a['input_ids'])

{'input_ids': tensor([[   2, 5891, 2205, 5971,   18,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
tensor([[   2, 5891, 2205, 5971,   18,    3]])


In [45]:
import torch.nn.functional as F

def inference(sentence, model):
    model.eval()
    MODEL_NAME = 'klue/bert-base'
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    output_pred = []
    output_prob = []
    
    # sentence 토크나이징
    tokenized_sent = tokenizing(sentence, tokenizer)
    
    
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent['input_ids'].to(device),
            attention_mask=tokenized_sent['attention_mask'].to(device),
            token_type_ids=tokenized_sent['token_type_ids'].to(device)
        )
    '''
    SequenceClassifierOutput(loss=None, logits=tensor([[ 2.8013,  0.9778, -2.0565,  2.7867, -1.6169, -2.9289]],
       device='cuda:0'), hidden_states=None, attentions=None)
    '''
    
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)    
    
    output_pred.append(result)
    output_prob.append(prob)
  
    return np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

In [56]:
def prediction(sentence):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForSequenceClassification.from_pretrained('./best_model')
    model.to(device)

    pred_answer, output_prob = inference(sentence, model)
    pred_answer = num_to_label(pred_answer)

    print(pred_answer, output_prob)

In [58]:
prediction('누군지 보려고 달려간 공주가 문을 열었을 때 어마나 세상에 문 앞에 어제 그 개구리가 앉아 있지 뭐예요.')
prediction('누가 봐도 공주의 얼굴이 붉으락푸르락한 게 보였어요.')
prediction('그래서 공주가 문을 사정없이 닫아버리고는 허둥대며 다시 자리로 와 식사를 계속했지만, 공주도 너무나도 놀란 건 사실이었어요.')
prediction('딸애야, 왜 그러니? 문 앞에서 너를 데려가려는 거인이라도 본 게냐?')
prediction('“아, 아냐,”라며 공주가 말했어요. “거인이 아니라 구역질나는 개구리 한 마리야.”')

['기쁨'] [[0.46331310272216797, 0.008690078742802143, 0.05478760227560997, 0.11694328486919403, 0.034610211849212646, 0.32165566086769104]]
['불안'] [[0.007257037330418825, 0.002930098446086049, 0.030506860464811325, 0.5612196922302246, 0.022893842309713364, 0.3751924932003021]]
['당황'] [[0.0023562866263091564, 0.992356538772583, 0.0004910796997137368, 0.004630960524082184, 9.798115934245288e-05, 6.713408947689459e-05]]
['당황'] [[0.00031427424983121455, 0.9889928698539734, 0.009809978306293488, 0.0005756760365329683, 0.00015612594143021852, 0.00015096743300091475]]
['분노'] [[0.0005168858915567398, 0.0013337378622964025, 0.7627708911895752, 0.0010220204712823033, 0.2239847034215927, 0.010371852666139603]]


In [78]:
span1 = '거대한 숲에서 아내와 함께 힘들게 살고 있는 나무꾼이 하나 있었습니다. 그들에게는 세 살 배기 어린 딸이 하나 있었습니다. 하지만 그들은 너무도 가난했기 때문에 양식을 구할 길이 없어 딸아이를 어찌 먹여살릴지 앞이 깜깜했습니다.'
span2 = '일곱 명의 새끼 염소들이 그 광경을 지켜보다 우물가로 달려와 환호성을 지르며 소리쳤어요. “만세! 늑대가 죽었다! 늑대가 죽었어!” 새끼 염소들은 너무 기뻐서 엄마 염소와 함께 우물가 주변을 둥글게 돌며 춤을 추었답니다.'
span3 = '왕은 어쩐지 제비가 두려웠다. 자신처럼 작은 이들에게는 거구의 새처럼 보였다. 하지만 엄지 공주를 보고는 무척 기뻐했다. 소녀는 지금껏 본 가장 아름다운 소녀였다. 그래서 황금 왕관을 벗어 소녀의 머리에 얹고는 이름을 물어도 되냐고 조심스럽게 묻고는 자신의 아내가 되어 달라고 부탁했다. 그러면 소녀는 모든 꽃의 여왕이 될 것이다. 정말이지 두꺼비 아들, 검은 벨벳 코트를 입은 두더지와는 완전히 다른 남편감이었다. 그래서 소녀는 이 매력적인 왕에게 “좋아요”라고 말했다. 꽃 속에서 자그마한 숙녀와 신사들이 나와 즐겁게 지켜보았다. 각자 엄지 공주에게 선물을 주었는데 최고의 선물은 커다란 은색 파리가 달았던 날개 한 쌍이었다. 그 날개를 등에 단단히 묶자 엄지 공주도 꽃과 꽃 사이를 살랑살랑 돌아다닐 수 있었다. 모두가 즐거워했다. 제비는 이들 위, 자기 둥지에 자리를 잡고는 제일 잘하는 노래를 들려주었다. 그렇지만 마음 깊은 곳에서는 슬픔이 밀려왔다. 엄지 공주를 무척이나 좋아해서 절대 헤어지고 싶지 않았기 때문이다.'


prediction(span1)
prediction(span2)
prediction(span3)

['상처'] [[0.0005434234044514596, 0.00031408053473569453, 7.354172703344375e-05, 0.0005094616208225489, 0.9984968900680542, 6.258022040128708e-05]]
['기쁨'] [[0.9994732737541199, 8.953684300649911e-05, 0.00020760462211910635, 8.277779852505773e-05, 4.648189860745333e-05, 0.0001003616489470005]]
['기쁨'] [[0.9990330934524536, 8.229914237745106e-05, 0.00012298690853640437, 0.0001815058058127761, 7.205220026662573e-05, 0.0005080334958620369]]
